http://fn.bmstu.ru/data-physics/library/physbook/tom5/ch4/texthtml/ch4_5.htm

In [ ]:
import Pkg
Pkg.add("LinearAlgebra")
Pkg.add("SparseArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Gnuplot")

In [ ]:
using LinearAlgebra, SparseArrays, BenchmarkTools
# using Plots
using Gnuplot

In [ ]:
garmosc(x) = x.^2

#const ħ = 6.5821e-16 # eV*s
const ħ = 1.0546e-34 # J*s
const m = 9.1094e-31 # kg
const q = 1.6022e-19 # Kl
C = ħ^2/m # m^-2 * J^-2

l = 10e-10 # метров - характерный масштаб задачи
dx = 0.005l
x1 = range(-2l, 2l, step = dx) |> collect
N = length(x1)

V = garmosc(x1)

@gp x1 V/q "with lines tit 'Parabola'"

In [ ]:
H = SymTridiagonal( ones(N), fill(-0.5, N-1) ) * C/dx^2;
Vm = diagm(V);

Ψ = eigvecs(H+Vm)
E = eigvals(H+Vm) / q
x1 /= l;

In [ ]:
yup = 8.0
n = 6
@gp x1 V/q "with lines tit 'Potential' lw 3" xlab="x, A" ylab="E, eV"
@gp :- "set title 'Garmonic Oscillator'" "set yrange [0:$yup]" 

for i = 1:n
    Y = abs2.(Ψ[:, i]) # вероятности
    Y /= 1.2maximum(Y)
    Y .+= E[i]
    @gp :- x1 Y "with lines tit 'E_$i = $(round(E[i], digits = 3)) eV' lw 2"
end

![](images/oscev.png)

## Аналитическое решение

$$
    \psi_n(x) = \frac{a^{1/4}}{\pi}\frac{1}{\sqrt{2^n n!}}H_n(y)e^{-y^2/2}
$$

$y = \sqrt a x$ 

$a = \frac{m}{\omega\hbar}$

$E_n = \hbar\omega(n + 1/2)$

Полином Эрмита:

$$H_n(x) = (-1)^n e^{x^2}\frac{d^n}{dx^n}e^{-x^2}$$

Нули на интервале $[-a,a], \ a = \sqrt{(n(n-1)/2)}$

$$
H_n^{phys}(x) = 2^{n/2}H_n^{math}(\sqrt 2 x)
$$

Рекурентная формула:

$$H_n(x) = \sum^{n/2}_{j=0}(-1)^j\frac{n!}{j!(n-2j)!}(2x)^{n-2j}$$

или детерминант:

$$
H_n(x) = \begin{vmatrix}
x & n-1 & 0 & 0 & \cdots  & 0\\ 
1 & x & n-2 & 0 & \cdots  & 0\\ 
0 & 1 & x & n-3 & \cdots  & 0\\ 
\cdots  & \cdots  & \cdots  & \cdots  & \cdots  & \cdots \\ 
 0& 0 & 0 & 0 & \cdots  & x
\end{vmatrix}
$$

In [ ]:
Hn(n, x) = sum( j-> (-1)^j*factorial(n)*(2x)^(n-2j)/factorial(j)/factorial(n-2j), 0:n÷2 )

# детерминант трехдиагональной матрицы
Htd(n::Int, x::Float64) = 2^(0.5n)*det( Tridiagonal(ones(n-1), fill(sqrt(2)*x,n), [1.0n-i for i=1:n-1] ) )

In [ ]:
@benchmark Hn(4, 20.)

In [ ]:
@benchmark Htd(4, 20.)

In [ ]:
@benchmark Htd(100,0.1)

In [ ]:
@benchmark Hn(BigInt(100),0.1)

In [ ]:
ω = 1/ħ

x2 = collect( range(-8, 8, step = 0.1) )

En(n) = (n + 0.5)*ω*ħ

function ψn(n, x)
    a = 1#m / (ω*ħ)
    y = sqrt(a)*x
    Hn(n, y) * exp(-0.5y^2) * a^0.25 / ( π * sqrt(2^n * factorial(n)) )
end

In [ ]:
yup = 10.0
n = 6
@gp x2 garmosc.(0.7x2) "with lines tit 'Potential' lw 3" xlab="x, A" ylab="E, eV"
@gp :- "set title 'Garmonic Oscillator'" "set yrange [0:$yup]" 

for i = 0:n
    Z = ψn.(i, x2)
    Z /= 2maximum(Z)
    Z .+= En(i)
    @gp :- x2 Z "with lines tit 'n = $i' lw 2"
end

![](images/garmoscan.png)